In [1]:
import gc
import pandas as pd
import numpy as np
import os

from tqdm import tqdm_notebook

pd.set_option('display.max_columns', 100)

from utils import loadpkl, to_feature, line_notify

%matplotlib inline

In [2]:
# load pkls
df_sales = loadpkl('../feats/sales.pkl')
df_calendar = loadpkl('../feats/calendar.pkl')
df_sell_prices = loadpkl('../feats/sell_prices.pkl')
df_sub = pd.read_csv('../input/sample_submission.csv')

In [15]:
df_sales.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,is_test
60034805,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,1
60034806,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,1
60034807,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,1
60034808,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,1
60034809,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1969,0,1


In [17]:
df_calendar.tail()

,date,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,seasonality
1964,2016-06-15,11620,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1,0.962513
1965,2016-06-16,11620,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0,0.967028
1966,2016-06-17,11620,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0,0.971257
1967,2016-06-18,11621,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0,0.975200
1968,2016-06-19,11621,2,6,2016,d_1969,13.0,0.0,3.0,0.0,0,0,0,0.978856


In [5]:
df_sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [6]:
df_sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df_sales['d_numeric'] = df_sales['d'].apply(lambda x: str(x)[2:]).astype(int)